# Kernel Methods

###### COMP4670/8600 - Introduction to Statistical Machine Learning - Tutorial 5

## Discussion

Get into groups of two or three and take turns explaining the following (about 2 minutes each):
- regression vs classification
- generative vs discriminative probabilistic methods
- logistic regression
- support vector machines
- basis functions vs kernels

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\dotprod}[2]{\langle #1, #2 \rangle}$

Setting up the environment

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

## The data set

This is the same dataset we used in Tutorial 3.

*We will use an old dataset on the price of housing in Boston (see [description](https://www.kaggle.com/vikrishnan/boston-house-prices)). The aim is to predict the median value of the owner occupied homes from various other factors. We will use a normalised version of this data, where each row is an example. The median value of homes is given in the first column (the label) and the value of each subsequent feature has been normalised to be in the range $[-1,1]$. Download this dataset from [mldata.org](http://mldata.org/repository/data/download/csv/housing_scale/). The column names are ['medv', 'crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat']*.

In [2]:
# Solution
names = ['medv', 'crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat']
data = np.loadtxt('housing_scale.csv', delimiter=',')

## Constructing new kernels

In the lectures, we saw that certain operations on kernels preserve positive semidefiniteness. Recall that a symmetric matrix $K\in \RR^n \times\RR^n$ is positive semidefinite if for all vectors $a\in\RR^n$ we have the inequality
$$
a^T K a \geqslant 0.
$$

Prove the following relations:
1. Given positive semidefinite matrices $K_1$, $K_2$, show that $K_1 + K_2$ is a valid kernel.
2. Given a positive semidefinite matrix $K$, show that $K^2 = K\cdot K$ is a valid kernel, where the multiplication is a pointwise multiplication (not matrix multiplication).

### Solution

1. We want to prove that
$$a^T (K_1 + K_2) a \geqslant 0.$$
By linearity of addition, distribute the multiplication of $a$
$$a^T (K_1 + K_2) a = a^T K_1 a^ + a^T K_2 a.$$
By the definition of kernels, $a^T K_1 a \geqslant 0$ and $a^T K_2 a \geqslant 0$ for all $a$. 
Since the sum of two non-negative numbers is non-negative, we have shown that $K_1+K_2$ is a valid kernel.

2. We may express $K$ in terms of its eigenvalues and eigenvectors as $K=\sum\limits_{i=1}^N\lambda_i\mathbf{u}_i\mathbf{u}_i^T$

where each $\lambda_i$ and $\mathbf{u}_i$ is an eigenvalue and eigenvector of $K$ (see equation 2.48 of Bishop).

Proof:

$K\mathbf{u}_i = \lambda_i \mathbf{u}_i$ by the definition of eigenvalues and eigenvectors

$K\mathbf{u}_i\mathbf{u}_i^T = \lambda_i \mathbf{u}_i\mathbf{u}_i^T$ multiplying both sides by $\mathbf{u}_i^T$

$K\sum\limits_{i=1}^N \mathbf{u}_i\mathbf{u}_i^T=\sum\limits_{i=1}^N\lambda_i\mathbf{u}_i\mathbf{u}_i^T$ sum over $N$ and move $K$ out of the summation

$KUU^T=\sum\limits_{i=1}^N\lambda_i\mathbf{u}_i\mathbf{u}_i^T$ where $U$ is a matrix whose columns are the eigenvectors of $K$

$K=\sum\limits_{i=1}^N\lambda_i\mathbf{u}_i\mathbf{u}_i^T$ because the columns form an orthonormal set, $U^TU=I$, $U^T=U^{-1}$ and so $UU^{-1}=UU^T=I$.

Now we move to $K \circ K$:

$K \circ K=\sum\limits_{i=1}^N\lambda_i\mathbf{u}_i\mathbf{u}_i^T \circ \sum\limits_{j=1}^N\lambda_j\mathbf{u}_j\mathbf{u}_j^T$

$=\sum\limits_{i=1}^N\sum\limits_{j=1}^N\lambda_i\lambda_j(\mathbf{u}_i\mathbf{u}_i^T) \circ (\mathbf{u}_j\mathbf{u}_j^T)$

$=\sum\limits_{i=1}^N\sum\limits_{j=1}^N\lambda_i\lambda_j(\mathbf{u}_i \circ \mathbf{u}_j)(\mathbf{u}_i \circ \mathbf{u}_j)^T$

Each matrix $(\mathbf{u}_i \circ \mathbf{u}_j)(\mathbf{u}_i \circ \mathbf{u}_j)^T$ is positive semi-definite. This is because for any vectors $a$ and $v$, $a^Tvv^Ta=a^Tv(a^Tv)^T=(a^Tv)^2\geq 0$. Because $K$ is positive semi-definite it has non-negative eigenvalues and so $\lambda_i\lambda_j\geq0$ for all $i,j$, so multiplying by these scalars still returns a positive semi-definite matrix. By the identity shown in part 1, we know that sums of positive semi-definite matrices are also positive semi-definite.

See https://en.wikipedia.org/wiki/Schur_product_theorem for this approach and other derivations of the same result.

## Polynomial kernel using closure

Using the properties proven above, show that the inhomogenous polynomial kernel of degree 2
$$k(\mathbf{x},\mathbf{x}') = (\dotprod{\mathbf{x}}{\mathbf{x}'} + 1)^2$$
is positive semidefinite.

### Solution

Consider the dot product $\dotprod{\mathbf{x}}{\mathbf{x}'}$ as the linear kernel.
\begin{align}
k(\mathbf{x},\mathbf{x}') &= (\dotprod{\mathbf{x}}{\mathbf{x}'} + 1)^2\\
    &= (\dotprod{\mathbf{x}}{\mathbf{x}'} + 1)(\dotprod{\mathbf{x}}{\mathbf{x}'} + 1)\\
    &= \dotprod{\mathbf{x}}{\mathbf{x}'}^2 + 2\dotprod{\mathbf{x}}{\mathbf{x}'} + 1.
\end{align}

This means that if we begin with a matrix $K'$ such that $K'_{ij} = \dotprod{\mathbf{x}}{\mathbf{x}'}$, we can construct matrix $K$ such that $K_{ij}=k(\mathbf{x},\mathbf{x}') = K' \circ K' + 2K' + \mathbf{1}$. Here $\mathbf{1}$ is the matrix of ones - it is possible to prove this is positive semi-definite by solving for its eigenvalues to show that they are non-negative, which is an equivalent condition. Since all of these matrices are positive semi-definite as shown above, their sum will also be positive semidefinite as we prevously showed.

## Empirical comparison

Recall from Tutorial 2 that we could explicitly construct the polynomial basis function. In fact this demonstrates the relation
$$
k(\mathbf{x},\mathbf{x}') = (\dotprod{\mathbf{x}}{\mathbf{x}'} + 1)^2 = \dotprod{\phi(\mathbf{x})}{\phi(\mathbf{x}')}.
$$
where
$$
\phi(\mathbf{x}) = (x_1^2, \ldots, x_n^2, \sqrt{2}x_{i} x_j {\forall i < j}, \sqrt{2}x_1, \ldots, \sqrt{2}x_n, 1)
$$
*This is sometimes referred to as an explicit feature map or the primal version of a kernel method.*

For the data above, construct two kernel matrices, one using the explicit feature map and the second using the equation for the polynomial kernel. Confirm that they are the same.

In [3]:
# Solution

def phi_quadratic(x):
    """Compute phi(x) for a single training example using quadratic basis function."""
    D = len(x)
    # Features are (1, {x_i}, {cross terms and squared terms})
    # Cross terms x_i x_j and squared terms x_i^2 can be taken from upper triangle of outer product of x with itself
    return np.hstack((1, np.sqrt(2)*x, np.sqrt(2)*np.outer(x, x)[np.triu_indices(D,k=1)], x ** 2))

def feature_map(X):
    """Return the matrix of the feature map."""
    num_ex, num_feat = X.shape
    Phi = np.zeros((num_ex, int(1+num_feat+num_feat*(num_feat+1)/2)))
    for ix in range(num_ex):
        Phi[ix,:] = phi_quadratic(X[ix,:])
    return Phi

def dotprod_quadratic(X):
    """Compute the kernel matrix using an explicit feature map of
    the inhomogeneous polynomial kernel of degree 2"""
    Phi = feature_map(X)
    return np.dot(Phi, Phi.T)

def kernel_quadratic(X,Y):
    """Compute the inhomogenous polynomial kernel matrix of degree 2"""
    lin_dot = np.dot(X,Y.T)
    dotprod = (lin_dot+1.)*(lin_dot + 1.)
    return dotprod

# Assume label is in the first column
X = np.array(data[:, 1:])
K = kernel_quadratic(X,X)
Kfeat = dotprod_quadratic(X)

print(K[:5,:5])
print(Kfeat[:5,:5])

[[ 60.94556031  55.59204065  57.88864391  58.94056732  58.3315753 ]
 [ 55.59204065  62.90868854  62.38586554  62.6226549   63.19323354]
 [ 57.88864391  62.38586554  66.67591107  68.75468564  67.95803214]
 [ 58.94056732  62.6226549   68.75468564  76.50307387  74.51105349]
 [ 58.3315753   63.19323354  67.95803214  74.51105349  73.4119558 ]]
[[ 60.94556031  55.59204065  57.88864391  58.94056732  58.3315753 ]
 [ 55.59204065  62.90868854  62.38586554  62.6226549   63.19323354]
 [ 57.88864391  62.38586554  66.67591107  68.75468564  67.95803214]
 [ 58.94056732  62.6226549   68.75468564  76.50307387  74.51105349]
 [ 58.3315753   63.19323354  67.95803214  74.51105349  73.4119558 ]]


Which method of computing the kernel matrix is faster? Discuss.

### Solution

* computing $k(\mathbf{x},\mathbf{x}')=(\dotprod{\mathbf{x}}{\mathbf{x}'} + 1)^2$ scales linearly with the length of $\mathbf{x}$
* computing $k(\mathbf{x},\mathbf{x}')=\dotprod{\phi(\mathbf{x})}{\phi(\mathbf{x}')}$ scales quadratically with the length of $\mathbf{x}$

## Regularized least squares with kernels

Show that the predictions using the regularized least squares solution can be expressed:
$$
y(\mathbf{x}) = \mathbf{k}(\mathbf{x})^T(\mathbf{K}+\lambda\mathbf{I})^{-1}\mathbf{t}
$$

where $\mathbf{K}=\mathbf{\Phi}\mathbf{\Phi}^T$ and the vector $\mathbf{k}(\mathbf{x})$ contains elements $k_n(\mathbf{x}) = k(\mathbf{x}_n,\mathbf{x})$.

Describe the reason for each step in your working.

### Solution

See Lecture 9. The important part is knowing the reason for each step.

Recall that the weights for the regularised least squares solution are given by $\mathbf{w} = \left( \lambda \mathbf{I} + \mathbf{\Phi}^T \mathbf{\Phi}\right)^{-1} \mathbf{\Phi}^T \mathbf{t}$. 

By substituting $w = \mathbf{\Phi}^T \mathbf{a}$, derive $\mathbf{a}$ in terms of the kernel matrix $\mathbf{K}$.

### Solution

$\mathbf{\Phi}^T \mathbf{a} = \left( \lambda \mathbf{I} + \mathbf{\Phi}^T \mathbf{\Phi}\right)^{-1} \mathbf{\Phi}^T \mathbf{t}$

$\mathbf{a} = (\mathbf{\Phi}^T)^{-1} \left( \lambda \mathbf{I} + \mathbf{\Phi}^T \mathbf{\Phi}\right)^{-1} \mathbf{\Phi}^T \mathbf{t}$

$\mathbf{a} = ( (\lambda \mathbf{I} + \mathbf{\Phi}^T \mathbf{\Phi})\mathbf{\Phi}^T)^{-1} \mathbf{\Phi}^T \mathbf{t}$

$\mathbf{a} = ( (\mathbf{\Phi}^T)^{-1}(\lambda \mathbf{I} + \mathbf{\Phi}^T \mathbf{\Phi})\mathbf{\Phi}^T)^{-1} \mathbf{t}$

$\mathbf{a} = ( \lambda \mathbf{I} + (\mathbf{\Phi}^T)^{-1}\mathbf{\Phi}^T \mathbf{\Phi}\mathbf{\Phi}^T)^{-1} \mathbf{t}$

$\mathbf{a} = ( \lambda \mathbf{I} + \mathbf{K})^{-1} \mathbf{t}$

## Comparing solutions in $a$ and $\mathbf{w}$

Implement the kernelized regularized least squares as above. 
*This is often referred to as the dual version of regularized least squares.*

Compare this with the solution from Tutorial 3. Implement two classes:
* ```RLSPrimal```
* ```RLSDual```

each which contain a ```train``` and ```predict``` function.

Think carefully about the interfaces to the training and test procedures for the two different versions of regularized least squares. Also think about the parameters that need to be stored in the class.

In [4]:
# Solution

class RLSPrimal(object):
    """Primal Regularized Least Squares"""
    def __init__(self, reg_param):
        self.reg_param = reg_param
        self.w = np.array([])   # This should be the number of features long
    
    def train(self, X, y):
        """Find the maximum likelihood parameters for the data X and labels y"""
        Phi = feature_map(X)
        num_ex = (Phi.T).shape[0]
        A = np.dot(Phi.T, Phi) + self.reg_param * np.eye(num_ex)
        b = np.dot(Phi.T, y)
        self.w = np.linalg.solve(A, b)
    
    def predict(self, X):
        """Assume trained. Predict on data X."""
        Phi = feature_map(X)
        return np.dot(Phi, self.w)

class RLSDual(object):
    def __init__(self, reg_param):
        self.reg_param = reg_param
        self.a = np.array([])    # This should be number of examples long

    def train(self, K, y):
        """Find the maximum likelihood parameters for the kernel matrix K and labels y"""
        num_ex = K.shape[0]
        A = K + self.reg_param * np.eye(num_ex)
        self.a = np.linalg.solve(A, y)
    
    def predict(self, K):
        """Assume trained. Predict on test kernel matrix K."""
        return np.dot(K, self.a)
    
    
def split_data(data):
    """Randomly split data into two equal groups"""
    np.random.seed(1)
    N = len(data)
    idx = np.arange(N)
    np.random.shuffle(idx)
    train_idx = idx[:int(N/2)]
    test_idx = idx[int(N/2):]

    # Assume label is in the first column
    X_train = data[train_idx, 1:]
    t_train = data[train_idx, 0]
    X_test = data[test_idx, 1:]
    t_test = data[test_idx, 0]
    
    return X_train, t_train, X_test, t_test

def rmse(label, prediction):
    N = len(label)
    return np.linalg.norm(label - prediction) / np.sqrt(N)

X_train, t_train, X_test, t_test = split_data(data)
P = RLSPrimal(1.1)
P.train(X_train, t_train)
pP = P.predict(X_test)

K_train = kernel_quadratic(X_train, X_train)
K_test = kernel_quadratic(X_test, X_train)   # This is not square
D = RLSDual(1.1)
D.train(K_train, t_train)
pD = D.predict(K_test)
print('RMSE: primal = %f, dual = %f' % (rmse(t_test, pP), rmse(t_test, pD)))

RMSE: primal = 3.830002, dual = 3.830002


What are the pros and cons of using the primal or dual methods for regularized least squares regression?

### Solution

* The kernel approach is computationally independent of the number of features. If the data $X$ is of size $N \times D$, then computing the kernel matrix $\mathbf{\Phi}\mathbf{\Phi}^T$ has cost $\mathcal{O}(N^2D)$ independent of the features used.
* The feature map approach has only a linear dependency on the number of examples. If the feature vector $\phi$ is of dimension $M$, then computing the matrix $\mathbf{\Phi}^T\mathbf{\Phi}$ has cost $O(M^2N)$.
* If $M$ is large relative to $N$ then the kernel approach is cheaper, otherwise the feature map approach is cheaper

## (optional) General kernel

Consider how you would generalise the two classes above if you wanted to have a polynomial kernel of degree 3. For the primal version, assume you have a function that returns the explicit feature map for the kernel ```feature_map(X)``` and for the dual version assume you have a function that returns the kernel matrix ```kernel_matrix(X)```.